In [5]:
cd ~/Documents/food/Food_project/

/home/serge/Documents/food/Food_project


In [6]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.metrics import roc_auc_score,confusion_matrix
from tqdm import tqdm

print(tf.__version__)

1.13.1


In [7]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
%env CUDA_VISIBLE_DEVICES=0,1
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)


sess = tf.Session(config=config)

set_session(sess)
tf.random.set_random_seed(123)

env: CUDA_VISIBLE_DEVICES=0,1


In [8]:
im_directory = "./Food_photos_copy"
fine_to_coarse = pd.read_csv("fine_to_coarse.csv",encoding = "utf-8")
coarse_labels = fine_to_coarse["Coarse names"]
coarse_labels = list(coarse_labels.to_numpy())

data = pd.read_pickle("./data.pkl")

In [9]:
coarse_labels

['Salad',
 'Soup',
 'Sandwich',
 'Wrap',
 'Drink',
 'Dressing',
 'Fruit',
 'Veggies',
 'Meats',
 'Pastry',
 'Rice',
 'Sushi',
 'Desserts/Sweet snacks',
 'Salty snacks',
 'Cereal/Cereal bar',
 'Pasta',
 'Yogurt',
 'Fried sides',
 'Pizza',
 'Burger',
 'Bread',
 'Cheese']

In [10]:
data.head()

,name,fine labels,coarse labels
0,C040.HT 2015-06-05 130853 C.png,"[Green beans with cherry tomatoes, Cornbread M...","[Veggies, Pasta, Desserts/Sweet snacks]"
1,A084.GK 2015-07-23 122234 A.png,"[Do Not Use Build a Chicken Salad, Glass Coke ...","[Sandwich, Fried sides, Bread, Salad, Drink]"
2,C060.PS 2016-06-03 132753 C.png,"[Build a Salad, Spring Mix Lettuce, Jalapenos,...","[Salad, Dressing]"
3,B113.TA 2015-08-13 132111 B.png,"[Meatballs, Roll, Just Cookies - White Chocola...","[Dressing, Desserts/Sweet snacks, Pastry, Meats]"
4,C017.WA 2016-05-10 123024 C.png,"[Build a Self Serve, Whole Banana, Fruit Salad]",[Fruit]


In [11]:
# split dataset into train & val
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.15,random_state=123)

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20, 
                                   zoom_range=[1.1,1.6],
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip = True,
                                  validation_split=0.20)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train, directory=im_directory, x_col='name', y_col='coarse labels',subset="training", target_size=(224, 224), color_mode='rgb', classes=coarse_labels, class_mode='categorical', batch_size=32, shuffle=True, seed=123, save_to_dir=None, save_prefix='', save_format='png', interpolation='nearest', drop_duplicates=True)
val_generator = train_datagen.flow_from_dataframe(train, directory=im_directory, x_col='name', y_col='coarse labels',subset="validation", target_size=(224, 224), color_mode='rgb', classes=coarse_labels, class_mode='categorical', batch_size=1, shuffle=False, seed=123, save_to_dir=None, save_prefix='', save_format='png', interpolation='nearest', drop_duplicates=True)
test_generator = test_datagen.flow_from_dataframe(test, directory=im_directory, x_col='name', y_col='coarse labels', target_size=(224, 224), color_mode='rgb', classes=coarse_labels, class_mode='categorical', batch_size=1, shuffle=False, seed=123, save_to_dir=None, save_prefix='', save_format='png', subset=None, interpolation='nearest', drop_duplicates=True)

Found 5250 images belonging to 22 classes.
Found 1312 images belonging to 22 classes.
Found 1159 images belonging to 22 classes.


# Training

In [10]:
#Model
from keras import Model
from keras.layers import Dense
from keras.applications.mobilenet_v2 import MobileNetV2

N_classes = 22
mobilenet = MobileNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(224,224,3), pooling='avg', classes=N_classes)

x = mobilenet.layers[-1].output
fc = Dense(N_classes,activation='sigmoid')(x)
model = Model(inputs=mobilenet.input, outputs=fc)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [13]:
from keras import metrics
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from time import time

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
name = "mobilenet_dense"
checkpointer = ModelCheckpoint(filepath=os.path.join("models",name+"_weights.hdf5"), 
                               monitor = 'val_loss',
                               verbose=1, 
                               save_best_only=True)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [14]:
# class Metrics(tf.keras.callbacks.Callback):
    
#     def __init__(self, val_data, batch_size = 1):
#         super().__init__()
#         self.validation_data = val_data
#         self.batch_size = batch_size
    
#     def on_train_begin(self, logs={}):
#         print(self.validation_data)
#         self.val_f1s = []
#         self.val_recalls = []
#         self.val_precisions = []
#         self.mean_auc_per_class = None
        
#     def on_epoch_end(self, epoch, logs={}):
#         batches = len(self.validation_data)
#         total = batches * self.batch_size

#         self.validation_data.reset()
#         val_pred = self.model.predict_generator(self.validation_data,steps = len(self.validation_data))
#         val_true = [[True if i in row else False for i in range(22)] for row in self.validation_data.classes]
#         val_true = np.array(val_true)
#         aucs = []
#         for i in range(22):
#             y_true = val_true[:,i]
#             y_score = val_pred[:,i]
#             aucs.append(roc_auc_score(y_true,y_score))
            
#         self.mean_auc_per_class = np.mean(aucs)
#         print("Mean per-class AUC:{}".format(self.mean_auc_per_class))
#         return

In [15]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEP_SIZE_TRAIN,
        epochs=200,
        validation_data=val_generator,
        validation_steps=STEP_SIZE_VALID,
        verbose= 1,
        shuffle = True,
        callbacks= [tensorboard,checkpointer])

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
164/164 [==============================] - 203s 1s/step - loss: 0.3430 - acc: 0.8608 - val_loss: 1.0030 - val_acc: 0.6978

Epoch 00001: val_loss improved from inf to 1.00300, saving model to models/mobilenet_dense_weights.hdf5
Mean per-class AUC:0.5139397102163411
Epoch 2/200
164/164 [==============================] - 191s 1s/step - loss: 0.3143 - acc: 0.8713 - val_loss: 1.1163 - val_acc: 0.7623

Epoch 00002: val_loss did not improve from 1.00300
Mean per-class AUC:0.5121182076453548
Epoch 3/200
164/164 [==============================] - 193s 1s/step - loss: 0.2999 - acc: 0.8781 - val_loss: 0.9756 - val_acc: 0.8144

Epoch 00003: val_loss improved from 1.00300 to 0.97562, saving model to models/mobilenet_dense_weights.hdf5
Mean per-class AUC:0.5218013166230718
Epoch 4/200
164/164 [==============================] - 192s 1s/step - loss: 0.2900 - acc: 0.8818 - val_loss: 0.9888 - val_acc: 0.7492

Epoch 00004: val_loss did not impro

164/164 [==============================] - 191s 1s/step - loss: 0.2123 - acc: 0.9130 - val_loss: 0.8666 - val_acc: 0.7879

Epoch 00034: val_loss did not improve from 0.72065
Mean per-class AUC:0.5606023338589627
Epoch 35/200
164/164 [==============================] - 192s 1s/step - loss: 0.2093 - acc: 0.9154 - val_loss: 0.8983 - val_acc: 0.8164

Epoch 00035: val_loss did not improve from 0.72065
Mean per-class AUC:0.5932770965407287
Epoch 36/200
164/164 [==============================] - 191s 1s/step - loss: 0.2103 - acc: 0.9145 - val_loss: 0.7796 - val_acc: 0.8159

Epoch 00036: val_loss did not improve from 0.72065
Mean per-class AUC:0.6126025345663513
Epoch 37/200
164/164 [==============================] - 191s 1s/step - loss: 0.2081 - acc: 0.9158 - val_loss: 0.8672 - val_acc: 0.8231

Epoch 00037: val_loss did not improve from 0.72065
Mean per-class AUC:0.5980546024501531
Epoch 38/200
164/164 [==============================] - 191s 1s/step - loss: 0.2120 - acc: 0.9140 - val_loss: 0.7

Mean per-class AUC:0.7237685728185912
Epoch 103/200
164/164 [==============================] - 191s 1s/step - loss: 0.1514 - acc: 0.9388 - val_loss: 0.3425 - val_acc: 0.8774

Epoch 00103: val_loss did not improve from 0.23200
Mean per-class AUC:0.6645620598732415
Epoch 104/200
164/164 [==============================] - 190s 1s/step - loss: 0.1411 - acc: 0.9417 - val_loss: 0.2751 - val_acc: 0.8950

Epoch 00104: val_loss did not improve from 0.23200
Mean per-class AUC:0.7414897542643692
Epoch 105/200
164/164 [==============================] - 190s 1s/step - loss: 0.1357 - acc: 0.9448 - val_loss: 0.2255 - val_acc: 0.9128

Epoch 00105: val_loss improved from 0.23200 to 0.22554, saving model to models/mobilenet_dense_weights.hdf5
Mean per-class AUC:0.7441696301117126
Epoch 106/200
164/164 [==============================] - 190s 1s/step - loss: 0.1428 - acc: 0.9412 - val_loss: 0.2338 - val_acc: 0.9127

Epoch 00106: val_loss did not improve from 0.22554
Mean per-class AUC:0.7225118901632498
E


Epoch 00138: val_loss did not improve from 0.22531
Mean per-class AUC:0.749464379212511
Epoch 139/200
164/164 [==============================] - 213s 1s/step - loss: 0.1096 - acc: 0.9556 - val_loss: 0.2743 - val_acc: 0.9134

Epoch 00139: val_loss did not improve from 0.22531
Mean per-class AUC:0.7490849675867935
Epoch 140/200
164/164 [==============================] - 214s 1s/step - loss: 0.1218 - acc: 0.9501 - val_loss: 0.7308 - val_acc: 0.8388

Epoch 00140: val_loss did not improve from 0.22531
Mean per-class AUC:0.576087927757375
Epoch 141/200
164/164 [==============================] - 213s 1s/step - loss: 0.1245 - acc: 0.9483 - val_loss: 0.3363 - val_acc: 0.8998

Epoch 00141: val_loss did not improve from 0.22531
Mean per-class AUC:0.6941947603684462
Epoch 142/200
164/164 [==============================] - 212s 1s/step - loss: 0.1138 - acc: 0.9534 - val_loss: 0.2948 - val_acc: 0.9040

Epoch 00142: val_loss did not improve from 0.22531
Mean per-class AUC:0.7056596794104947
Epoch 14

164/164 [==============================] - 213s 1s/step - loss: 0.0848 - acc: 0.9665 - val_loss: 0.2474 - val_acc: 0.9214

Epoch 00175: val_loss did not improve from 0.22531
Mean per-class AUC:0.7618477853206734
Epoch 176/200
164/164 [==============================] - 214s 1s/step - loss: 0.0894 - acc: 0.9631 - val_loss: 0.2611 - val_acc: 0.9180

Epoch 00176: val_loss did not improve from 0.22531
Mean per-class AUC:0.7558662867485869
Epoch 177/200
164/164 [==============================] - 214s 1s/step - loss: 0.0818 - acc: 0.9662 - val_loss: 0.2931 - val_acc: 0.9123

Epoch 00177: val_loss did not improve from 0.22531
Mean per-class AUC:0.7484533795795518
Epoch 178/200
164/164 [==============================] - 213s 1s/step - loss: 0.0816 - acc: 0.9665 - val_loss: 0.2551 - val_acc: 0.9224

Epoch 00178: val_loss did not improve from 0.22531
Mean per-class AUC:0.7722760060005491
Epoch 179/200
164/164 [==============================] - 213s 1s/step - loss: 0.0827 - acc: 0.9659 - val_loss:

In [16]:
with open('models/'+name+'_architecture.json', 'w') as f:
    f.write(model.to_json())